In [2]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [3]:
# from pathlib import Path
# input_dir = Path(parms["dir_proc_data"])
# ipath = Path(input_dir / "SYNTHETIC_dataset_dict")
# ipath

In [4]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

# Import Data

In [5]:
lDataNames = ['T81','T85','DT']
dataDir = '../Data/DataV2/TTS/'

In [6]:
dataName = 'T81'

In [7]:
df_tr = load_dataset_to_df(f"{dataDir}{dataName}_Train.arrow", verbose=True)
df_ts = load_dataset_to_df(f"{dataDir}{dataName}_Test.arrow", verbose=True)
df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)

In [8]:
df_full = pd.concat([df_tr,df_ts,df_h])
lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']

for df in [df_tr,df_ts,df_h,df_full]:
    df.drop(df[df["LC_STATUS"] == "HC"].index, inplace=True)  # drop HC
    df["LC_STATUS"] = df["LC_STATUS"].apply(
        lambda x: 1 if x == "LC_POS" else 0
    )  # Convert to 0==LCNeg, 1==LCPos
    df.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

In [9]:
df_tr

,sex,age,MED-Analgesics,MED-Anesthetics,MED-Antibacterials,MED-Anticonvulsants,MED-Antidepressants,MED-Antifungals,MED-Antimigraine-agents,MED-Antiparasitics,...,HHX-OTHER-genetic-disorder,HHX-OTHER-hyper,HHX-OTHER-hypo,HHX-OTHER-resp,HHX-OTHER-sickle-cell-anemia,HHX-OTHER-surgery,HHX-OTHER-ulcerative-colitis,SUBST-1,SUBST-5,LC_STATUS
0,2.0,44.0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0.0,0.0,0
1,1.0,75.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,1
2,1.0,37.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
3,2.0,68.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0.0,1
4,2.0,40.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2809,1.0,43.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2810,1.0,70.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0.0,0
2811,1.0,70.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,1
2812,1.0,72.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [10]:
# saveDir = MPutils.get_saving_dir(f"OUTPUT/MP/05-classifiers/DataV2/{dataName}/")


lClassifiers = ["RF", "LogR",'XGBcpu'] # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
lCV = [5] # different cross-validation splits
lTrees=[100,1000] #num trees
lMD=[3,6,10,15, None] #max depth

lBrt_MD = [3, 5, 7]
brt_trees = 1000
brt_iter = 500
brt_thres = 100
brt_toprank = 5


# FIXME - BELOW CODE FOR TESTING PURPOSES

# lClassifiers = [
#     "RF",
#     "LogR",
#     "XGBcpu",
# ]  # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
# lCV = [5]  # different cross-validation splits
# lTrees = [10]  # num trees
# lMD = [3]  # max depth

# lBrt_MD = [3]
# brt_trees=10
# brt_iter = 10
# brt_thres=100
# brt_toprank=5

In [63]:
lRunDataFrames = []
for dataName in tqdm(lDataNames,desc='DataSet Main Outer Loop'):

    saveDir = MPutils.get_saving_dir(
        f"OUTPUT/MP/05-classifiers/DataV2/{dataName}/"
    )

    ## HOLDOUT SET ONLY
    x_h, y_h = getXY(df_h)

    dfRHold = classify_MP(
        X=x_h,
        y=y_h,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
    )
    dfRHold["Title"] = f"{dataName}_Holdout"

    dfRHold.to_csv(f"{saveDir}CA_{dataName}_Holdout.csv", index=False)

    ## Full SET ONLY
    x_F, y_F = getXY(df_full)

    dfRFull = classify_MP(
        X=x_F,
        y=y_F,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
    )
    dfRFull["Title"] = f"{dataName}_Full"

    dfRFull.to_csv(f"{saveDir}CA_{dataName}_Full.csv", index=False)

    ## BORUTA Feature Selection

    x_Tr, y_Tr = getXY(df_tr)
    for boruta_md in tqdm(lBrt_MD[:],desc='Boruta MD Loop'):
        # run brouta = get true and top feat
        trueFeat, topFeat = boruta_fs(
            X=x_Tr.values,
            y=y_Tr,
            feat_list=x_Tr.columns,
            trees=brt_trees,
            ittr=brt_iter,
            threshold=brt_thres,
            top_rank=brt_toprank,
            verbose=0,
            model=RandomForestClassifier(
                n_jobs=-1, class_weight="balanced", max_depth=boruta_md, random_state=42
            ),
            fileName=f"{saveDir}FS_{dataName}_Boruta_T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}_MD{boruta_md}.csv",
        )

        ## Test Brouta Features
        x_TsMain, y_Ts = getXY(df_ts)

        x_Ts_True = x_TsMain[trueFeat]
        x_Ts_Top = x_TsMain[topFeat]

        # True Feat
        dfR_True = classify_MP(
            X=x_Ts_True,
            y=y_Ts,
            lScorers=lScorersBinary,
            lClassifiers=lClassifiers,
            lTrees=lTrees,
            lMD=lMD,
            lCV=lCV,
        )
        dfR_True["Title"] = f"{dataName}_Boruta_True"
        dfR_True['brt_nTrue'] = len(trueFeat)
        dfR_True['brt_nTop'] = len(topFeat)
        # Top Feat
        dfR_Top = classify_MP(
            X=x_Ts_Top,
            y=y_Ts,
            lScorers=lScorersBinary,
            lClassifiers=lClassifiers,
            lTrees=lTrees,
            lMD=lMD,
            lCV=lCV,
        )

        dfR_Top["Title"] = f"{dataName}_Boruta_Top"
        dfR_Top["brt_md"] = boruta_md

        dfR_True['brt_nTrue'] = len(trueFeat)
        dfR_True["brt_nTop"] = len(topFeat)

    dfCLFRun = pd.concat([dfRHold, dfRFull, dfR_True, dfR_Top])

    dfCLFRun["MainDataset"] = dataName
    dfCLFRun["date"] = datetime.today()
    dfCLFRun["brt_params"] = (
        f"Boruta_T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}"
    )

    dfCLFRun = MPutils.reorder_columns(dfCLFRun, ["MainDataset", "Title", "date",'brt_nTrue','brt_nTop'])

    dfCLFRun.to_csv(f"{saveDir}CLFRun_{dataName}_Results.csv", index=False)
    lRunDataFrames.append(dfCLFRun)

DataSet Main Outer Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

In [71]:
dfCombRes = pd.concat(lRunDataFrames)

In [72]:
dfCombRes["classifier"] = dfCombRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])

In [73]:
lColFirst = [
    "MainDataset",
    "Title",
    "date",
    "classifier",
    "cv",
    "brt_nTrue",
    "brt_nTop",
]

In [74]:
dfCombRes = MPutils.reorder_columns(dfCombRes, lColFirst)

In [75]:
dfCombRes.to_csv('OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined.csv',index=False)

In [76]:
lColsSel = lColFirst+[x for x in dfCombRes if x.startswith('mean_test') or x.startswith('param_') or x.startswith('brt')]

In [77]:
dfCombRes[lColsSel].to_csv(
    "OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_SelCol.csv", index=False
)

# Holdoutset - validation testing - Boruta Tested Best Feature

In [ ]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lClassifiers = ["RF", "ET", "LogR"] # different classifiers
lCV = [3,5,10] # different cross-validation splits
lTrees=[100,1000,10000] #num trees
lMD=[None] #max depth


In [33]:
selFeat = "Test_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True_LCN_LCP".replace(
    "_LCN_LCP", ""
).replace("Test_", "")
lSelFeat = dFeat[selFeat]
x,y = getXY(T81_h)
x=x[lSelFeat]

In [47]:
dfResults = classify_MP(X=x, y=y, lScorers=lScorersBinary, lClassifiers=lClassifiers, lTrees=lTrees, lMD=lMD,lCV=lCV)

In [ ]:
# dfResults.columns

In [55]:
dfResults['Title']='R81_Holdout_'+selFeat

In [56]:
lResCol = ['Title','cv']+[x for x in dfResults.columns if x.startswith("param_") or x.startswith("mean_test_")]

In [71]:
dfResults[lResCol] # all results

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,param_clf__max_iter,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
0,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,100,42,NaN,0.702455,0.699885,0.788684,0.674349,0.665841,0.665841,0.733930,0.684176,0.718477,0.684176
1,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,1000,42,NaN,0.701390,0.698875,0.791620,0.673046,0.665793,0.665793,0.731958,0.681562,0.718642,0.681562
2,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,RandomForestClassifier(),None,10000,42,NaN,0.705681,0.703180,0.791043,0.677596,0.670422,0.670422,0.735938,0.686724,0.722897,0.686724
3,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,100,42,NaN,0.692784,0.688157,0.730954,0.653469,0.626392,0.626392,0.749922,0.685168,0.700019,0.685168
4,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,1000,42,NaN,0.694935,0.690631,0.728423,0.657253,0.633337,0.633337,0.747926,0.685721,0.703843,0.685721
5,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,ExtraTreesClassifier(),None,10000,42,NaN,0.696010,0.691629,0.727802,0.658020,0.633337,0.633337,0.749922,0.687405,0.704433,0.687405
6,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,3,LogisticRegression(),NaN,NaN,NaN,1000,0.770128,0.765683,0.840141,0.739753,0.705355,0.705355,0.826011,0.778178,0.764789,0.778178
0,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,100,42,NaN,0.716405,0.713311,0.798193,0.685627,0.670623,0.670623,0.756000,0.706254,0.729044,0.706254
1,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,1000,42,NaN,0.725007,0.721461,0.797399,0.692943,0.672922,0.672922,0.770000,0.717786,0.733738,0.717786
2,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),None,10000,42,NaN,0.723943,0.720786,0.797618,0.693693,0.677573,0.677573,0.764000,0.712914,0.734625,0.712914


In [39]:
dfResults[
    (dfResults["cv"] == 5)
    # & (dfResults["param_clf"] == "RandomForestClassifier()")
    & (dfResults["param_clf__n_estimators"] == 1000)
][lResCol].T

,8,11
Title,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True
cv,5,5
param_clf,RandomForestClassifier(),ExtraTreesClassifier()
param_clf__max_depth,NaN,NaN
param_clf__n_estimators,1000.0,1000.0
param_clf__random_state,42.0,42.0
mean_test_accuracy,0.725007,0.715336
mean_test_balanced_accuracy,0.721461,0.709272
mean_test_roc_auc,0.797399,0.751675
mean_test_f1,0.692943,0.669664


In [38]:
dfResults[(dfResults['cv']==5) & (dfResults['param_clf']=='RandomForestClassifier()') & (dfResults['param_clf__n_estimators']==1000)][lResCol]

,Title,cv,param_clf,param_clf__max_depth,param_clf__n_estimators,param_clf__random_state,mean_test_accuracy,mean_test_balanced_accuracy,mean_test_roc_auc,mean_test_f1,mean_test_recall,mean_test_sensitivity,mean_test_specificity,mean_test_precision,mean_test_NPV,mean_test_PPV
8,R81_Holdout_T81_Boruta_T1000_itrr200_thres100_topR5_MD7_True,5,RandomForestClassifier(),NaN,1000.0,42.0,0.725007,0.721461,0.797399,0.692943,0.672922,0.672922,0.77,0.717786,0.733738,0.717786


In [79]:
dfResults.to_csv(
    f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}.csv", index=False
)
dfResults[lResCol].to_csv(f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}_selCol.csv", index=False)

In [37]:
dfResults = pd.read_csv(
    f"OUTPUT/MP/05-classifiers/Validation/CA_{'R81_Holdout_'+selFeat}.csv"
)

### Feature Importance of Best Set


In [61]:
clf = RandomForestClassifier(n_estimators=1000,max_depth=None,random_state=42)
clf.fit(x,y)
feature_importances = clf.feature_importances_
feature_importances = pd.Series(feature_importances, index=x.columns).sort_values(
    ascending=False
)

In [62]:
feature_importances

age                                     0.419838
dag                                     0.235372
SYMPT-fatigue___2                       0.084360
SYMPT-dizziness___2                     0.079019
SYMPT-muscle_aches_myalgia___2          0.074982
SYMPT-shortness_of_breath_dyspne___2    0.059716
SYMPT-joint_pain_arthralgia___2         0.046712
dtype: float64

In [76]:
feature_importances.to_csv(f"OUTPUT/MP/05-classifiers/Validation/FR_{'R81_Holdout_'+selFeat}.csv", index=False)

age                                     0.419838
dag                                     0.235372
SYMPT-fatigue___2                       0.084360
SYMPT-dizziness___2                     0.079019
SYMPT-muscle_aches_myalgia___2          0.074982
SYMPT-shortness_of_breath_dyspne___2    0.059716
SYMPT-joint_pain_arthralgia___2         0.046712

In [65]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)
cross_val_score(clf, x, y, cv=5, scoring="accuracy").mean()

0.7250071876257836